In [3]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


In [4]:
%pip install -qU langchain-google-vertexai

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from langchain_google_vertexai import ChatVertexAI
import vertexai

vertexai.init(project="project_id")     #Add project id
llm = ChatVertexAI(model="gemini-1.5-flash")

In [14]:
import getpass
import os

if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = "mistral_api_key"   #Add mistral api key

from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-large-latest")

In [7]:
%pip install -qU langchain-core

In [8]:
from langchain_core.embeddings import DeterministicFakeEmbedding

embeddings = DeterministicFakeEmbedding(size=4096)

In [9]:
%pip install -qU langchain-chroma

In [10]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)

In [22]:
%pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.5 MB/s eta 0:00:00


In [39]:
import bs4
from langchain import hub
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

loader=PyPDFLoader('pdf_name.pdf') #Enter PDF
docs = loader.load()
print(docs)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

[Document(metadata={'source': '/content/the_secret_history_of_the_mongols_the_life_and_times_of_chinggis_khan1.pdf', 'page': 50, 'page_label': '44'}, page_content='[14] Dobun-mergen loaded up the three-year-old deer.\nWhile continuing on his way, he met a poor man leading\nhis young son.\n[15] Dobun-mergen asked: ‘Who are you?’ The man\nreplied: ‘I am a man called Ma’aliq of the Baya’uds and in\ndistress. I will give you this son of mine in exchange for\npart of that beast’s flesh.’\n[16] On hearing those words, Dobun-mergen tore off one\nof the deer’s hind-legs and gave it to the man. He then took\nthe boy back to his yurt, where the boy worked as a servant.\n[17] Not long afterwards, Dobun-mergen died. After his\ndeath, despite the loss of her husband, Alan Qo’a bore\nthree more sons called Buqu-qadagi, Buqatu-salji, and\nBodonchar-mungqaq.94\n[18] Belgünütei and Bügünütei, the two sons born earlier\nto Dobun-mergen, talked together about their mother Alan\nQo’a behind her back: This

In [38]:
response = graph.invoke({"question": "What is the story about?"})
print(response["answer"])

I don't know. The provided context discusses two different topics: a historical narrative involving Chinggis Qahan and a description of a scientific discovery agent called ChemCrow, but it does not give a clear storyline for a single narrative.
